## Import libraries

In [ ]:
import tensorflow as tf
import warnings
warnings.filterwarnings("ignore")
import keras
from tensorflow.keras.models import Model,load_model
from tensorflow.keras import Input
from tensorflow.keras.layers import concatenate,Dense,Flatten,Dropout
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
import tensorflow.keras.callbacks as kcallbacks
from tensorflow.keras.applications.xception import  Xception
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.optimizers.legacy import Adam #hadas
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
import os
import math
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras.optimizers import SGD
import operator
import numpy as np
from PIL import Image
from collections import defaultdict
from google.colab import drive
import pandas as pd
import os.path
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

##google drive

In [ ]:
PROJECT_PATH= '/content/drive/MyDrive/project/'

In [ ]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


##Unzip data

In [ ]:
DATA1='NSL-KDD'
# DATA1='CICIDS2017'

In [ ]:
def unzip_folder(name_data,train_flag=True):
  if train_flag:
    data_zip_path=os.path.join(PROJECT_PATH,'data',name_data,'train_224_'+name_data )
    print(data_zip_path)
  else:
    data_zip_path=os.path.join(PROJECT_PATH,'data',name_data,'test_224_'+name_data )
  print(data_zip_path)
  !unzip -n {data_zip_path}



###Unzip train

In [ ]:
unzip_folder(DATA1)
# os.rename('train_224', 'train_224_'+DATA1)

Streaming output truncated to the last 5000 lines.
  inflating: train_224_NSL-KDD/4/49357.png  
  inflating: train_224_NSL-KDD/4/49359.png  
  inflating: train_224_NSL-KDD/4/4936.png  
  inflating: train_224_NSL-KDD/4/49361.png  
  inflating: train_224_NSL-KDD/4/49362.png  
  inflating: train_224_NSL-KDD/4/49363.png  
  inflating: train_224_NSL-KDD/4/49364.png  
  inflating: train_224_NSL-KDD/4/49365.png  
  inflating: train_224_NSL-KDD/4/49367.png  
  inflating: train_224_NSL-KDD/4/49368.png  
  inflating: train_224_NSL-KDD/4/49369.png  
  inflating: train_224_NSL-KDD/4/4937.png  
  inflating: train_224_NSL-KDD/4/49370.png  
  inflating: train_224_NSL-KDD/4/49372.png  
  inflating: train_224_NSL-KDD/4/49373.png  
  inflating: train_224_NSL-KDD/4/49374.png  
  inflating: train_224_NSL-KDD/4/49375.png  
  inflating: train_224_NSL-KDD/4/49376.png  
  inflating: train_224_NSL-KDD/4/49377.png  
  inflating: train_224_NSL-KDD/4/49378.png  
  inflating: train_224_NSL-KDD/4/49379.png  
  infl

###Unzip test

In [ ]:
unzip_folder(DATA1,train_flag=False)
# os.rename('test_224', 'test_224_'+DATA1)

Streaming output truncated to the last 5000 lines.
  inflating: test_224_NSL-KDD/4/33168.png  
  inflating: test_224_NSL-KDD/4/33182.png  
  inflating: test_224_NSL-KDD/4/33183.png  
  inflating: test_224_NSL-KDD/4/33184.png  
  inflating: test_224_NSL-KDD/4/33185.png  
  inflating: test_224_NSL-KDD/4/33187.png  
  inflating: test_224_NSL-KDD/4/33190.png  
  inflating: test_224_NSL-KDD/4/33194.png  
  inflating: test_224_NSL-KDD/4/33196.png  
  inflating: test_224_NSL-KDD/4/3320.png  
  inflating: test_224_NSL-KDD/4/33200.png  
  inflating: test_224_NSL-KDD/4/33202.png  
  inflating: test_224_NSL-KDD/4/33204.png  
  inflating: test_224_NSL-KDD/4/33207.png  
  inflating: test_224_NSL-KDD/4/33209.png  
  inflating: test_224_NSL-KDD/4/33217.png  
  inflating: test_224_NSL-KDD/4/33220.png  
  inflating: test_224_NSL-KDD/4/33222.png  
  inflating: test_224_NSL-KDD/4/33225.png  
  inflating: test_224_NSL-KDD/4/33240.png  
  inflating: test_224_NSL-KDD/4/33243.png  
  inflating: test_224_NSL-

##CUDA

In [ ]:
gpus=tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(gpus))
if len(gpus):
  print(gpus)

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


##Constants

In [ ]:
#generate images from train set and validation set
TARGET_SIZE=(224,224)
INPUT_SIZE=(224,224,3)
BATCHSIZE=128


## Generate Training and Test Images

###Generate Functions

In [ ]:
def make_name_folder(data):
  train_folder='./train_224_'+data+'/'
  test_folder='./test_224_'+data+'/'
  return train_folder,test_folder

def image_generator(data):
  train_folder,test_folder =make_name_folder(data)
  #Normalization
  train_datagen = ImageDataGenerator(rescale=1./255)
  test_datagen = ImageDataGenerator(rescale=1./255)
  train_generator = train_datagen.flow_from_directory(
        train_folder,
        target_size=TARGET_SIZE,
        # color_mode=COLOR_MODE,
        # shuffle=False,
        class_mode='categorical',
        batch_size=BATCHSIZE,)
  test_generator = test_datagen.flow_from_directory(
        test_folder,
        target_size=TARGET_SIZE,
        # color_mode=COLOR_MODE,
        batch_size=BATCHSIZE,
        shuffle=False,
        class_mode='categorical')
  return train_generator,test_generator

###Genrate NSL-KDD

In [ ]:
train_generator,test_generator=image_generator(DATA1)

Found 224352 images belonging to 5 classes.
Found 56087 images belonging to 5 classes.


# label

In [ ]:
def make_labels(generator):
  labels = []
  number_of_examples = len(generator.filenames)
  number_of_generator_calls = math.ceil(number_of_examples / (1.0 * BATCHSIZE))
  for _ in range(0,int(number_of_generator_calls)):
    img, label = generator.next()
    row,col=label.shape
    for i in range(int(row)):
      labels.append(np.where(label[i]==1.)[0][0])

  return labels


In [ ]:
train_label=make_labels(train_generator)

In [ ]:
len(train_label)

224352

In [ ]:
test_label=make_labels(test_generator)

In [ ]:
len(test_label)

56087

## Load 5 trained CNN models

In [ ]:

def make_path_file(data,dir,file):
  path1=os.path.join('training',data,dir,file)
  path=PROJECT_PATH+path1
  return path

def load_df(data):
  # path1=os.path.join('training',data,'best_model','best_model.csv')
  path=make_path_file(data,'best_model','best_model.csv')
  df=pd.read_csv(path)
  df=df.set_index('model')
  return df

def make_path_model(name_model,df,data):
  path=make_path_file(data,'best_model',df.loc[name_model ,'name file'])
  return path


def load_best_model(name_model,df,data):
  path=make_path_model(name_model,df,data)
  return load_model(path)

In [ ]:
best_model_df=load_df(DATA1)

In [ ]:
best_model_df

,name file
model,
Inception,inception_frozen:40_epochs=20_patience=3_lr=0....
InceptionResnet,inceptionresnet_frozen:506_epochs=20_patience=...
VGG16,VGG16_frozen:16_epochs=20_patience=4_lr=0.002_...
VGG19,VGG19_frozen:18_epochs=15_patience=3_lr=0.001_...
Xception,xception_frozen:129_epochs=15_patience=2_lr=0....


In [ ]:
path=make_path_model('Xception',best_model_df,DATA1)
path

'/content/drive/MyDrive/project/training/NSL-KDD/best_model/xception_frozen:129_epochs=15_patience=2_lr=0.002_dropout=0.4.h5'

In [ ]:
 #load model 1: xception
xception_model=load_best_model('Xception',best_model_df,DATA1)

In [ ]:
 #load model 2: VGG16
vgg_model=load_best_model('VGG16',best_model_df,DATA1)

In [ ]:
 #load model 3: VGG19
vgg19_model=load_best_model('VGG19',best_model_df,DATA1)

In [ ]:
 #load model 4: inception
incep_model=load_best_model('Inception',best_model_df,DATA1)

In [ ]:
 #load model 5: inceptionresnet
inres_model=load_best_model('InceptionResnet',best_model_df,DATA1)

## Use the original CNN base models to make predictions

### function

In [ ]:
def make_df_predictions(models,generator,labels,verbose = 0):
  df_pred=pd.DataFrame()
  for model in models:
    y_pred = model.predict_generator(generator,verbose = verbose)
    df_model = pd.DataFrame(data=y_pred)
    df_pred=pd.concat([df_pred,df_model], axis=1)
  df_pred['label']=labels
  return df_pred




def calculate_performance(model, test_generator,test_labels,verbose = 0):
  test_generator.reset()
  y_pred = model.predict(test_generator,batch_size=BATCHSIZE,verbose = verbose)
  y_pred_bool = np.argmax(y_pred, axis=1)
  print(classification_report(test_labels, y_pred_bool))
  print('Accuracy score: ',accuracy_score(test_labels, y_pred_bool))



def performance_evaluation_model(model,test_generator,test_labels,verbose = 0):
  # df = pd.DataFrame(data=y_pred)
  test_generator.reset()
  results =model.evaluate(test_generator, batch_size=BATCHSIZE,verbose = verbose)
  print(f'evaluate: accuracy={results[1]}, loss= {results[0]}')
  calculate_performance(model,test_generator,test_labels,verbose = verbose)
  # print(classification_report(test_generator.labels, y_pred_bool))
  # return df

# def change_layer_name(model,name):
#   for i, layer in enumerate(model.layers):
#     layer._name  = layer.name+name


In [ ]:
performance_evaluation_model(xception_model,test_generator,test_label,verbose = 1)

439/439 [==============================] - 107s 244ms/step - loss: 0.0851 - accuracy: 0.9641
evaluate: accuracy=0.9640914797782898, loss= 0.0851123109459877
439/439 [==============================] - 107s 243ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     15284
           1       0.99      0.99      0.99     10711
           2       0.93      0.89      0.91     10113
           3       0.99      0.99      0.99      9938
           4       0.90      0.94      0.92     10041

    accuracy                           0.96     56087
   macro avg       0.96      0.96      0.96     56087
weighted avg       0.96      0.96      0.96     56087

Accuracy score:  0.964091500704263


In [ ]:
performance_evaluation_model(vgg_model,test_generator,test_label,verbose = 1)

439/439 [==============================] - 111s 245ms/step - loss: 0.0805 - accuracy: 0.9623
evaluate: accuracy=0.9623085856437683, loss= 0.08050175756216049
439/439 [==============================] - 107s 243ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     15284
           1       0.99      0.99      0.99     10711
           2       0.95      0.86      0.90     10113
           3       0.99      0.99      0.99      9938
           4       0.87      0.96      0.91     10041

    accuracy                           0.96     56087
   macro avg       0.96      0.96      0.96     56087
weighted avg       0.96      0.96      0.96     56087

Accuracy score:  0.9623085563499563


In [ ]:
performance_evaluation_model(vgg19_model,test_generator,test_label,verbose = 1)

439/439 [==============================] - 108s 244ms/step - loss: 0.0823 - accuracy: 0.9634
evaluate: accuracy=0.9634496569633484, loss= 0.0823143944144249
439/439 [==============================] - 107s 243ms/step
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     15284
           1       0.99      0.99      0.99     10711
           2       0.90      0.93      0.91     10113
           3       0.99      0.99      0.99      9938
           4       0.93      0.90      0.91     10041

    accuracy                           0.96     56087
   macro avg       0.96      0.96      0.96     56087
weighted avg       0.96      0.96      0.96     56087

Accuracy score:  0.9634496407367126


In [ ]:
performance_evaluation_model(incep_model,test_generator,test_label,verbose = 1)

439/439 [==============================] - 111s 246ms/step - loss: 0.0658 - accuracy: 0.9713
evaluate: accuracy=0.9712945818901062, loss= 0.06580214947462082
439/439 [==============================] - 108s 244ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     15284
           1       1.00      0.99      0.99     10711
           2       0.89      0.98      0.93     10113
           3       0.99      0.99      0.99      9938
           4       0.98      0.89      0.93     10041

    accuracy                           0.97     56087
   macro avg       0.97      0.97      0.97     56087
weighted avg       0.97      0.97      0.97     56087

Accuracy score:  0.9712945958956621


In [ ]:
performance_evaluation_model(inres_model,test_generator,test_label,verbose = 1)

439/439 [==============================] - 113s 247ms/step - loss: 0.0775 - accuracy: 0.9639
evaluate: accuracy=0.9638953804969788, loss= 0.07752110809087753
439/439 [==============================] - 111s 245ms/step
              precision    recall  f1-score   support

           0       1.00      0.99      0.99     15284
           1       1.00      0.99      0.99     10711
           2       0.94      0.87      0.91     10113
           3       0.99      0.99      0.99      9938
           4       0.88      0.96      0.92     10041

    accuracy                           0.96     56087
   macro avg       0.96      0.96      0.96     56087
weighted avg       0.96      0.96      0.96     56087

Accuracy score:  0.9638953768252893


## make df

In [ ]:
test_pred=make_df_predictions([xception_model,incep_model,inres_model],test_generator,test_label,verbose = 0)

In [ ]:
path=make_path_file(DATA1,'predictions','test_pred.csv')
test_pred.to_csv(path)

In [ ]:
train_pred=make_df_predictions([xception_model,incep_model,inres_model],train_generator,train_label,verbose = 0)

In [ ]:
path=make_path_file(DATA1,'predictions','train_pred.csv')
train_pred.to_csv(path)

In [ ]:
from sklearn.metrics import accuracy_score
def acc_test(model,generator,labels):
  pred = model.predict_generator(generator, verbose=1)
  y_pred=pred.argmax(axis=1)
  print(accuracy_score(labels, y_pred ))

In [ ]:
acc_test(xception_model,test_generator,test_label)

439/439 [==============================] - 106s 241ms/step
0.20550216627739049
